In [28]:
import pandas as pd
import numpy as np
import sqlite3

In [29]:
partners_data_set = pd.read_excel('data_files/Partners Data Set_ref.xlsx')
indirect_data_set = pd.read_excel('data_files/outcome/temp.xlsx')

In [30]:
conn2 = sqlite3.connect('data_files/customer_data.db')
query = "SELECT * FROM customers"  
df_customers = pd.read_sql_query(query, conn2)
conn2.close()

In [31]:
temp = indirect_data_set[indirect_data_set['type'] != 'Agent']

In [32]:
mapping_dict = temp.dropna().set_index('sold_to_customer')['customer_name'].to_dict()

partners_data_set['customer_name'] = ''
partners_data_set['customer_name'].replace('', np.nan, inplace=True)

# Fill NaN values in "for_bu" column using the mapping dictionary
partners_data_set['customer_name'] = partners_data_set['customer_name'].fillna(partners_data_set['Customer Number'].map(mapping_dict))

In [33]:
# Get DataFrame where Column1 contains NaN values
filtered_df = partners_data_set[partners_data_set['customer_name'].isna()]
filled_df = partners_data_set[~partners_data_set['customer_name'].isna()]

In [34]:
temp2 = df_customers[(df_customers['agent_person'].notna()) & (df_customers['agent_person'] != 'None')]

In [ ]:
filtered_df['Customer Number'] = filtered_df['Customer Number'].astype('str')
temp2['agent_person'] = temp2['agent_person'].astype('int')
temp2['agent_person'] = temp2['agent_person'].astype('str')

temp2 = temp2[['agent_person', 'customer_name']]
mapping_dict = temp2.dropna().set_index('agent_person')['customer_name'].to_dict()
filtered_df['customer_name'] = filtered_df['customer_name'].fillna(filtered_df['Customer Number'].map(mapping_dict))
filtered_df['customer_name'] = filtered_df['customer_name'].fillna(filtered_df['Customer Name'])

In [38]:
merged = pd.concat([filtered_df, filled_df])
merged = merged[['customer_name', 'Fiscal Year', 'Order Intake']]
merged = merged.rename(columns={'Fiscal Year':'FY', 'Order Intake':'order_intake_amount_eur'})
merged = merged[merged['FY'].isin([2017, 2018, 2019, 2020, 2021, 2022,
       2023])]
merged_sum = merged.groupby(['customer_name', 'FY']).sum().reset_index()

indirect_data_set = indirect_data_set[indirect_data_set['tier_new'] == 'Channel Partner']
indirect_data_set_cut = indirect_data_set[['customer_name', 'FY', 'order_intake_amount_eur']]
indirect_data_set_cut = indirect_data_set_cut.dropna(subset=['customer_name'])
indirect_data_set_cut['FY'] = indirect_data_set_cut['FY'].dt.year
indirect_data_set_cut_sum = indirect_data_set_cut.groupby(['customer_name', 'FY']).sum().reset_index()

In [58]:
final_df = merged_sum.merge(indirect_data_set_cut_sum, on=['customer_name', 'FY'], how='outer')

# Keep rows where the ratio of column1 to column2 is in the range 0.98 - 1.02
#final_df = final_df[~(final_df['order_intake_amount_eur_x'] / final_df['order_intake_amount_eur_y']).between(0.99, 1.01)]

writer = pd.ExcelWriter('data_files/outcome/yorick_comparison.xlsx')
# Save each DataFrame to a separate sheet in the same file
final_df.to_excel(writer, sheet_name='target', index=False)
# Save the file
writer.close()

In [49]:
pivot_table1 = partners_data_set.pivot_table(columns='Fiscal Year', values='Order Intake', aggfunc='sum')
pivot_table2 = indirect_data_set_cut_sum.pivot_table(columns='FY', values='order_intake_amount_eur', aggfunc='sum')

In [55]:
writer = pd.ExcelWriter('data_files/outcome/two_pivots.xlsx')
# Save each DataFrame to a separate sheet in the same file
pivot_table1.to_excel(writer, sheet_name='target', index=False)
pivot_table2.to_excel(writer, sheet_name='target2', index=False)
# Save the file
writer.close()